In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

# Load data

### Barcodes

In [3]:
with open('allele_dic_with_WT.pkl', 'rb') as fobj:
    barcodes_map = pickle.load(fobj)

In [4]:
barcode_len = len(next(iter(barcodes_map)))
nbarcodes = len(barcodes_map)

In [5]:
barcodes = np.empty((nbarcodes, barcode_len), dtype='i1')

for i, bc in enumerate(barcodes_map):
    barcodes[i] = np.frombuffer(bc.encode('ascii'), dtype='i1')

### Genetic code

In [6]:
with open('translate.pkl', 'rb') as fobj:
    codon_to_aa = pickle.load(fobj)

In [7]:
dna_codon_to_aa = {codon.replace('U', 'T'): aa for codon, aa in codon_to_aa.items()}

# Distance funcs

In [8]:
import numba as nb

In [9]:
@nb.jit('int8(int8[:], int8[:])', nopython=True)
def hamming(a, b):
    l = a.shape[0]
    d = 0

    for i in range(l):
        if a[i] != b[i]:
            d += 1
            
    return d

In [10]:
@nb.jit('void(int8[:], int8[:, :], int8[:])', nopython=True)
def hamming_row(a, b, out):
    l = a.shape[0]
    n = b.shape[0]
    
    assert b.shape[1] == l
    assert out.shape[0] == n
    
    for i in range(n):
        for j in range(l):
            if a[j] != b[i, j]:
                out[i] += 1

### Time it

In [12]:
out = np.zeros(nbarcodes, dtype='i1')

# Calculate distance matrix

In [17]:
from tqdm import tqdm

In [18]:
dists = np.zeros((nbarcodes, nbarcodes), dtype='i1')

In [19]:
dists[:] = 0

for i in tqdm(range(nbarcodes)):
    hamming_row(barcodes[i], barcodes, dists[i])

100%|██████████| 31485/31485 [01:00<00:00, 521.66it/s]


In [ ]:
assert np.min(dists) == 0

In [ ]:
max_dist = np.max(dists)

# Stats

# Model

In [ ]:
from scipy.stats import binom

In [ ]:
dist_distr = binom(barcode_len, .75)

In [ ]:
dist_k = np.arange(barcode_len + 1)

In [ ]:
dist_pmf = dist_distr.pmf(dist_k)
dist_cdf = dist_distr.cdf(dist_k)

### Bin all pairwise dists

In [ ]:
dist_counts = np.bincount(dists.flat, minlength=barcode_len)
dist_counts[0] = 0
dist_counts //= 2

In [ ]:
npairs = dist_counts.sum()

In [ ]:
dists_cum = np.cumsum(dist_counts)

In [ ]:
plt.loglog(dist_cdf * npairs, dists_cum, marker='o')
plt.plot(*[[dist_cdf[1] * npairs, npairs]] * 2, linestyle='--')

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
plt.figure(figsize=(12, 6))

_x = np.arange(1, max_dist + 1)
plt.plot(_x, dist_pmf[1:] * npairs, marker='o', label='Predicted')
plt.plot(_x, dist_counts[1:], marker='o', label='Actual')

plt.yscale('log')
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.title('Distribution of barcode pairwise Hamming distances')
plt.legend()

pass

$$ \begin{align} P(X = k) &= \binom{n}{k} p^k (1 - p)^{n - k} \\ n &= 18 \\ p &= \frac{3}{4} \end{align}$$

### Minimum distances

In [ ]:
mindists = np.asarray([np.delete(dists[i], i).min() for i in range(nbarcodes)])

In [ ]:
mindist_counts = np.bincount(mindists, minlength=barcode_len + 1)
mindist_cum = np.cumsum(mindist_counts)

In [ ]:
min_cdf = 1 - dist_distr.sf(dist_k) ** (nbarcodes - 1)

In [ ]:
min_pmf = np.zeros(barcode_len + 1)
min_pmf[0] = min_cdf[0]
min_pmf[1:] = np.diff(min_cdf)
min_pmf.sum()

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(np.arange(1, 8), min_pmf[1:8] * nbarcodes, marker='o', label='Predicted')
plt.plot(np.arange(1, 8), mindist_counts[1:8], marker='o', label='Actual')

plt.yscale('log')

plt.legend()
plt.title('Distribution of barcode nearest-neighbor hamming distances')

pass

### Means

In [ ]:
meandists = dists.sum(axis=1) / (nbarcodes - 1)

In [ ]:
plt.hist(meandists, bins=50)
pass

In [ ]:
mediandists = np.median(dists, axis=1)

In [ ]:
plt.hist(mediandists, bins=50)
pass

# Neighbors

In [ ]:
neighbor_counts = np.asarray([np.sum(row == 1) for row in dists])

In [ ]:
plt.hist(neighbor_counts, bins=50, log=True)
pass

In [ ]:
neighbor_pair_count = neighbor_counts.sum() // 2
neighbor_pair_count

In [ ]:
neighbors = np.asarray([(i, j) for i in range(nbarcodes) for j in np.flatnonzero(dists[i] == 1) if i < j])

assert len(neighbors) == neighbor_pair_count

In [ ]:
neighbors

In [ ]:
neighbor_components = []

for b1, b2 in neighbors:
    
    for i, c1 in enumerate(neighbor_components):
        if b1 in c1:
            break
    else:
        i = None
        c1 = None
        
    for j, c2 in enumerate(neighbor_components):
        if b2 in c2:
            break
    else:
        j = None
        c2 = None
        
    if i is None and j is None:
        neighbor_components.append({b1, b2})
        
    elif i is None and j is not None:
        c2.update([b1, b2])
        
    elif j is None and i is not None:
        c1.update([b1, b2])
        
    elif i == j:
        c1.update([b1, b2])
        
    else:
        # Merge components
        c1.update(c2)
        c1.update([b1, b2])
        del neighbor_components[j]

In [ ]:
len(neighbor_components)

In [ ]:
plt.hist(list(map(len, neighbor_components)), log=True)
pass

In [ ]:
has_neighbor = set.union(*neighbor_components)

In [ ]:
len(has_neighbor)

In [ ]:
mutations_by_index = dict()

for i, bcnums in enumerate(barcodes):
    bc = bc_nums_to_str(bcnums)
    res, codon = barcodes_map[bc]
    
    aa = None if codon == 'WT' else dna_codon_to_aa[codon]
    mutations_by_index[i] = (res, aa)

In [ ]:
from collections import Counter

In [ ]:
neighbor_res_counts = np.bincount([res for res, aa in (mutations_by_index[i] for i in has_neighbor)])

In [ ]:
all_res_counts = np.bincount([res for res, aa in mutations_by_index.values()])

In [ ]:
plt.plot(neighbor_res_counts[2:] / all_res_counts[2:-1])

In [ ]:
neighbor_aa_counts = Counter([aa for res, aa in (mutations_by_index[i] for i in has_neighbor)])

In [ ]:
all_aa_counts = Counter([res for res, aa in mutations_by_index.values()])

In [ ]:
c = max(neighbor_components, key=len)

In [ ]:
c

In [ ]:
barcodes

In [ ]:
def bc_nums_to_str(nums):
    return ''.join(map(chr, nums))

In [ ]:
for i in c:
    print(color_seq(bc_nums_to_str(barcodes[i])))

In [ ]:
esc = '\x1b'

In [ ]:
def color_seq(seq):
    cols = dict(a='31', c='34', g='32', t='33')
    s2 = []
    
    for c in seq:
        try:
            col = cols[c.lower()]
        except KeyError:
            s2.append(c)
            continue
            
        s2.append('\x1b[{}m{}\x1b[0m'.format(col, c))
        
    return ''.join(s2)

In [ ]:
print(color_seq(bc_nums_to_str(barcodes[0])))